## NOWCASTING APRIL USING LINEAR REGRESSION
* This approach assumes that Zindi has released the cpi data whhich happens approximately 24th of each month


In [ ]:
#this is just for getting the file paths if you have them, no need to do this
!git clone https://github.com/koleshjr/Koding_With_Kolesh.git


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler


import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


path = '/content/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv(path + 'CPI_Historic_Values_Zindi_Mar_23.csv')
vehicles = pd.read_csv(path + 'Naamsa_Vehicle_Sales.csv')

In [ ]:
cpi.head()

NameError: ignored

In [ ]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)
cpi_pivot


### Simple EDA


In [ ]:


# Create a temporary dataframe with Month and Year columns
df_temp = cpi_pivot.copy()
df_temp['Year'] = pd.DatetimeIndex(df_temp['Month']).year
df_temp['Month'] = pd.DatetimeIndex(df_temp['Month']).month

# Define the categories to plot
categories_to_plot = cpi_pivot.columns[1:]

# Create subplots
num_categories = len(categories_to_plot)
num_rows = math.ceil(num_categories / 2)
num_cols = 2

fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 10))
fig.tight_layout(pad=3.0)

# Iterate over each category and plot its seasonality
for i, category in enumerate(categories_to_plot):
    row = i // num_cols
    col = i % num_cols

    ax = axs[row, col] if num_rows > 1 else axs[col]

    sns.pointplot(x='Month', y=category, hue='Year', data=df_temp, ax=ax)
    ax.set_title(category)
    ax.set_xlabel('Month')
    ax.set_ylabel('Value')

# Hide any unused subplots
for i in range(num_categories, num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols

    ax = axs[row, col] if num_rows > 1 else axs[col]
    ax.axis('off')

plt.suptitle('Seasonality of Categories', y=1.05)
plt.show()


### Add a row for April
* so this is one of the things that keeps on changing across mini challenge: just the date_str

In [ ]:
date_str = '2023-04-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_pivot

### Additional data source

* Vehicles

In [ ]:
import datetime
cpi_pivot['year_month'] = pd.to_datetime(cpi_pivot['Month'], format='%Y-%b').dt.strftime('%Y-%m')
start_date = datetime.datetime.strptime("2020-12-31", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-03-31", "%Y-%m-%d")

# difference between each date. M means one month end
D = 'M'

date_list = pd.date_range(start_date, end_date, freq=D)[::-1]
vehicles['Date'] = date_list
vehicles['Date'] = pd.to_datetime(vehicles['Date'], format='%Y-%b-%d')
vehicles['year_month'] = pd.to_datetime(vehicles['Date'], format='%Y-%b').dt.strftime('%Y-%m')

cpi_pivot = cpi_pivot.merge(vehicles[['year_month', 'Total_Local Sales', 'Total_Export_Sales']], on='year_month', how='left')



* Indices

* Currencies

* Additional data source from the links provided:

### Feature Engineering

In time series feature engineering, "lagging" means looking at past values of a variable (such as temperature, sales, or stock prices) to understand its patterns or make predictions. By creating lagged features, you can include information from previous time points in your analysis. This can help you find trends, detect patterns, or forecast what might happen in the future based on what happened in the past.

* Find your sweet spot


In [ ]:
feats_to_lag = [col for col in cpi_pivot.columns if col not in ['Month', 'year_month']]
for col in feats_to_lag:
  for i in range(1, 8):
    cpi_pivot[f"prev_{i}_month_{col}"]= cpi_pivot[col].shift(i)

cpi_pivot

In [ ]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

In [ ]:
train = cpi_pivot[cpi_pivot['Month'] != '2023-04-30']
test = cpi_pivot[cpi_pivot['Month'] == '2023-04-30']

training_set = train[train['Month']!= '2023-03-31']
validation_set = train[train['Month']== '2023-03-31']

train.shape, test.shape, training_set.shape, validation_set.shape

### Training and Validation

In [ ]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month', 'year_month','Total_Local Sales', 'Total_Export_Sales']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]


lr_models = {}
y_pred = []
scaler = MinMaxScaler()

#training
for target_col in target_cols:
  lr_model = LinearRegression()
  X_train_scaled = scaler.fit_transform(X_train)
  lr_model.fit(X_train_scaled, y_train[target_col])
  lr_models[target_col] = lr_model

#validation

for target_col in target_cols:
  lr_model = lr_models[target_col]
  X_val_scaled = scaler.transform(X_val)
  y_pred_col = lr_model.predict(X_val_scaled)
  y_pred.append(y_pred_col)


#scoring

y_pred = np.array(y_pred).T

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})

#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))

print(f'RMSE: {rmse}')







### Training and Inference

In [ ]:

X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]


lr_models = {}
y_pred = []
scaler = MinMaxScaler()

#training
for target_col in target_cols:
  lr_model = LinearRegression()
  X_train_scaled = scaler.fit_transform(X_train)
  lr_model.fit(X_train_scaled, y_train[target_col])
  lr_models[target_col] = lr_model

#validation

for target_col in target_cols:
  lr_model = lr_models[target_col]
  X_val_scaled = scaler.transform(X_val)
  y_pred_col = lr_model.predict(X_val_scaled)
  y_pred.append(y_pred_col)

y_pred = np.array(y_pred).T


### Prepare the Submission

In [ ]:
def prepSub(y_pred: list, target_cols: list, test: pd.DataFrame, prefix: str) -> pd.DataFrame:
    # Create a submission DataFrame
    submission_df = pd.DataFrame(y_pred, columns=target_cols)

    # Add the month column to the submission DataFrame
    submission_df['Month'] = test['Month']

    # Set the month column as the index
    submission_df.set_index('Month', inplace=True)

    # Add the prefix to the column names
    # Add the prefix to the column names
    submission_df.columns = [prefix + '_' + col.lower().replace('_', ' ').strip() for col in submission_df.columns]
    submission_df = submission_df.rename(columns = {f'{prefix}_headline cpi': f'{prefix}_headline CPI'})


    # Reshape the DataFrame using pd.melt()
    submission_df = pd.melt(submission_df.reset_index(), id_vars=['Month'], var_name='ID', value_name='Value')


    return submission_df[['ID', 'Value']]

#also the prefix here changes across mini challenges
submission = prepSub(y_pred, target_cols, test, 'April')
submission


,ID,Value
0,April_alcoholic beverages and tobacco,110.206050
1,April_clothing and footwear,103.762447
2,April_communication,99.742120
3,April_education,110.440096
4,April_food and non-alcoholic beverages,117.845937
5,April_headline CPI,109.590150
6,April_health,110.155237
7,April_household contents and services,108.087269
8,April_housing and utilities,104.375742
9,April_miscellaneous goods and services,109.413577


In [ ]:
submission.to_csv("April_5.csv", index = False)